# 4 initialize

change ${CORE_ROOT} to your path of core.

In [ ]:
export CORE_ROOT="${HOME}/core"

change ${PJ_ROOT} to your path of uoa-poc2.

In [ ]:
export PJ_ROOT="${HOME}/uoa-poc2"
cd ${PJ_ROOT};pwd

example)
```
/Users/user/uoa-poc2
```

## load environment variables

load from `core`

In [ ]:
source ${CORE_ROOT}/docs/environments/azure_aks/env

load from `uoa-poc2`

In [ ]:
source ${PJ_ROOT}/docs/environments/azure_aks/env

## setup alias

In [ ]:
alias now="python -c 'import datetime; print(datetime.datetime.now().strftime(\"%Y-%m-%dT%H:%M:%S.%f+09:00\"))'"
alias iso8601="python -c 'import datetime; print(datetime.datetime.now(tz=datetime.timezone.utc).isoformat(timespec=\"seconds\"))'"

## prepare

### prepare a command to subscribe all topics

In [ ]:
echo "mosquitto_sub -h mqtt.${DOMAIN} -p 8883 --cafile ${CORE_ROOT}/secrets/DST_Root_CA_X3.pem -d -u iotagent -P ${MQTT__iotagent} -t /#"

### subscribe all topics

_Outside of this notebook_
1. open a ternminal.
1. run the above command displayed `prepare a command to subscribe all topics`.

### start uoa_poc2_bridge

_On ROS Kinect PC_
1. open three ternminal.
1. start roscore by `roscore` (terminal 1)
1. (megarover) start uoa_poc2_bridge by `roslaunch uoa_poc2_bridge uoa_poc2_bridge_mr.launch` (terminal 2)  
(turtlebot3) start uoa_poc2_bridge by `roslaunch uoa_poc2_bridge uoa_poc2_bridge_tb3.launch` (terminal 2)

* install python packages by `rosdep install --from-paths src --ignore-src -r -y` if missing packages

### show a ros message for sending a command to delivery_robot_01

_On ROS Kinect PC_
1. start a terminal
1. watch the ros topic by `rostopic echo /robot_bridge/turtlebot_01/cmd`

### show a ros message for sending a command to delivery_robot_02

_On ROS Kinect PC_
1. start a terminal
1. watch the ros topic by `rostopic echo /robot_bridge/megarover_01/cmd`

### show log messages of `robot-controller`

_Outside of this notebook_
1. open a ternminal.
1. run below command to show log messages of `robot-controller`
> `kubectl logs -f -l app=robot-controller`

## initialize robot entities

In [ ]:
DELIVERY_ROBOT_IDS=$(cat << __EOD__
[
  {
    "id": "delivery_robot_01"
  },
  {
    "id": "delivery_robot_02"
  }
]
__EOD__
)

TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
LEN=$(echo ${DELIVERY_ROBOT_IDS} | jq length)
count=0
if [ ${LEN} -gt 0 ]; then
  for i in $(seq 0 $((${LEN} - 1))); do
    id=$(echo ${DELIVERY_ROBOT_IDS} | jq .[${i}].id -r)
    echo "${id} will be initialized."
    curl -i -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${DELIVERY_ROBOT_SERVICEPATH}" -H "Content-Type: application/json" https://api.${DOMAIN}/orion/v2/entities/${id}/attrs?type=${DELIVERY_ROBOT_TYPE} -X PATCH -d @- <<__EOS__
{
  "mode": {
    "value": "standby",
    "type": "string"
  },
  "remaining_waypoints_list": {
    "type": "array",
    "value": []
  },
  "navigating_waypoints": {
    "type": "object",
    "value": {}
  },
  "current_routes": {
    "type": "array",
    "value": []
  },
  "order": {
    "type": "object",
    "value": {}
  },
  "current_mode": {
    "type": "string",
    "value": " ",
    "metadata": {}
  },
  "current_state": {
    "type": "string",
    "value": " ",
    "metadata": {}
  },
  "caller": {
    "type": "string",
    "value": " ",
    "metadata": {}
  },
  "last_processed_time": {
    "type": "ISO8601",
    "value": "1970-01-01T09:00:00+09:00",
    "metadata": {}
  }
}
__EOS__
  let ++count
  done
fi
echo "${count} entities were initialized."

example)

```
delivery_robot_01 will be initialized.
HTTP/1.1 204 No Content
content-length: 0
fiware-correlator: 48b434b0-01f9-11ea-b2b0-3e52937c8752
date: Fri, 08 Nov 2019 07:27:56 GMT
x-envoy-upstream-service-time: 6
server: envoy

delivery_robot_02 will be initialized.
HTTP/1.1 204 No Content
content-length: 0
fiware-correlator: 48d049f2-01f9-11ea-acd2-aaa36e462053
date: Fri, 08 Nov 2019 07:27:56 GMT
x-envoy-upstream-service-time: 9
server: envoy

2 entities were initialized.
```

### confirm entities

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${DELIVERY_ROBOT_SERVICEPATH}" "https://api.${DOMAIN}/orion/v2/entities/?type=${DELIVERY_ROBOT_TYPE}&attrs=mode,order,remaining_waypoints_list,navigating_waypoints,current_routes,current_mode,current_state,caller,last_processed_time" | jq .

example)

```json
[
  {
    "id": "delivery_robot_01",
    "type": "delivery_robot",
    "mode": {
      "type": "string",
      "value": "standby",
      "metadata": {}
    },
    "order": {
      "type": "object",
      "value": {},
      "metadata": {}
    },
    "remaining_waypoints_list": {
      "type": "array",
      "value": [],
      "metadata": {}
    },
    "navigating_waypoints": {
      "type": "object",
      "value": {},
      "metadata": {}
    },
    "current_routes": {
      "type": "array",
      "value": [],
      "metadata": {}
    },
    "current_mode": {
      "type": "string",
      "value": "standby",
      "metadata": {
        "TimeInstant": {
          "type": "datetime",
          "value": "2019-11-21T13:10:41.009+09:00"
        }
      }
    },
    "current_state": {
      "type": "string",
      "value": "standby",
      "metadata": {
        "TimeInstant": {
          "type": "datetime",
          "value": "2019-11-21T13:10:41.522+09:00"
        }
      }
    },
    "caller": {
      "type": "string",
      "value": " ",
      "metadata": {}
    },
    "last_processed_time": {
      "type": "ISO8601",
      "value": "1970-01-01T00:00:00.00Z",
      "metadata": {}
    }
  },
  {
    "id": "delivery_robot_02",
    "type": "delivery_robot",
    "mode": {
      "type": "string",
      "value": "standby",
      "metadata": {}
    },
    "order": {
      "type": "object",
      "value": {},
      "metadata": {}
    },
    "remaining_waypoints_list": {
      "type": "array",
      "value": [],
      "metadata": {}
    },
    "navigating_waypoints": {
      "type": "object",
      "value": {},
      "metadata": {}
    },
    "current_routes": {
      "type": "array",
      "value": [],
      "metadata": {}
    },
    "current_mode": {
      "type": "string",
      "value": "standby",
      "metadata": {
        "TimeInstant": {
          "type": "datetime",
          "value": "2019-11-21T13:10:41.521+09:00"
        }
      }
    },
    "current_state": {
      "type": "string",
      "value": "standby",
      "metadata": {
        "TimeInstant": {
          "type": "datetime",
          "value": "2019-11-21T13:10:41.686+09:00"
        }
      }
    },
    "caller": {
      "type": "string",
      "value": " ",
      "metadata": {}
    },
    "last_processed_time": {
      "type": "ISO8601",
      "value": "1970-01-01T00:00:00.00Z",
      "metadata": {}
    }
  }
]
```

## initialize token entities

In [ ]:
TOKEN_IDS=$(cat << __EOD__
[
  {
    "id": "token_a"
  },
  {
    "id": "token_b"
  },
  {
    "id": "token_c"
  }
]
__EOD__
)

TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
LEN=$(echo ${TOKEN_IDS} | jq length)
count=0
if [ ${LEN} -gt 0 ]; then
  for i in $(seq 0 $((${LEN} - 1))); do
    id=$(echo ${TOKEN_IDS} | jq .[${i}].id -r)
    echo "${id} will be initialized."
    curl -i -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${TOKEN_SERVICEPATH}" -H "Content-Type: application/json" https://api.${DOMAIN}/orion/v2/entities/${id}/attrs?type=${TOKEN_TYPE} -X PATCH -d @- <<__EOS__
{
  "is_locked": {
    "type": "boolean",
    "value": false
  },
  "lock_owner_id": {
    "type": "string",
    "value": ""
  },
  "waitings": {
    "type": "array",
    "value": []
  }
}
__EOS__
  let ++count
  done
fi
echo "${count} entities were initialized."

example)

```
token_a will be initialized.
HTTP/1.1 204 No Content
content-length: 0
fiware-correlator: f20a7ef2-04d8-11ea-acd2-aaa36e462053
date: Mon, 11 Nov 2019 23:14:00 GMT
x-envoy-upstream-service-time: 6
server: envoy

...

token_c will be initialized.
HTTP/1.1 204 No Content
content-length: 0
fiware-correlator: f2331a10-04d8-11ea-bfd1-2621012d3c7f
date: Mon, 11 Nov 2019 23:14:00 GMT
x-envoy-upstream-service-time: 5
server: envoy

3 entities were initialized.
```

### confirm entities

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${TOKEN_SERVICEPATH}" "https://api.${DOMAIN}/orion/v2/entities/?type=${TOKEN_TYPE}" | jq .

example)

```json
[
  {
    "id": "token_a",
    "type": "token",
    "TimeInstant": {
      "type": "ISO8601",
      "value": "2019-11-08T05:06:42.00Z",
      "metadata": {}
    },
    "is_locked": {
      "type": "boolean",
      "value": false,
      "metadata": {}
    },
    "lock_owner_id": {
      "type": "string",
      "value": "",
      "metadata": {}
    },
    "waitings": {
      "type": "array",
      "value": [],
      "metadata": {}
    }
  },
  {
    "id": "token_b",
    "type": "token",
    "TimeInstant": {
      "type": "ISO8601",
      "value": "2019-11-08T05:06:42.00Z",
      "metadata": {}
    },
    "is_locked": {
      "type": "boolean",
      "value": false,
      "metadata": {}
    },
    "lock_owner_id": {
      "type": "string",
      "value": "",
      "metadata": {}
    },
    "waitings": {
      "type": "array",
      "value": [],
      "metadata": {}
    }
  },
  {
    "id": "token_c",
    "type": "token",
    "TimeInstant": {
      "type": "ISO8601",
      "value": "2019-11-08T05:06:42.00Z",
      "metadata": {}
    },
    "is_locked": {
      "type": "boolean",
      "value": false,
      "metadata": {}
    },
    "lock_owner_id": {
      "type": "string",
      "value": "",
      "metadata": {}
    },
    "waitings": {
      "type": "array",
      "value": [],
      "metadata": {}
    }
  }
]
```

## initialize physical robots

### move delivery_robot_01 to staret position

In [ ]:
mosquitto_pub -h mqtt.${DOMAIN} -p 8883 --cafile ${CORE_ROOT}/secrets/DST_Root_CA_X3.pem -d -u iotagent -P ${MQTT__iotagent} -t /${DELIVERY_ROBOT_TYPE}/${DELIVERY_ROBOT_01}/cmd -m "{\"send_cmd\":{\"time\":\"$(now)\",\"cmd\":\"navi\",\"waypoints\":[{\"point\":{\"x\":-0.4,\"y\":1.5,\"z\":0},\"angle\":{\"roll\":0,\"pitch\":0,\"yaw\":-1.57}}]}}"

#### confirm mqtt message

example)

```
Client mosq-TX4bePNymjLv3bvrtN received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_01/cmd', ... (159 bytes))
{"send_cmd":{"time":"2019-11-11T12:13:41.824398+09:00","cmd":"navi","waypoints":[{"point":{"x":-0.4,"y":1.5,"z":0},"angle":{"roll":0,"pitch":0,"yaw":-1.57}}]}}
```

#### confirm ros message for delivery_robot_01 ( `/robot_bridge/turtlebot_01/cmd` )

example)

```yaml
id: "delivery_robot_01"
type: "delivery_robot"
time: "2019-11-11T12:13:41.824398+09:00"
cmd: "navi"
waypoints:
  -
    point:
      x: -0.4
      y: 1.5
      z: 0.0
    angle_optional:
      valid: True
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: -1.57
---
```

### move delivery_robot_02 to staret position

In [ ]:
mosquitto_pub -h mqtt.${DOMAIN} -p 8883 --cafile ${CORE_ROOT}/secrets/DST_Root_CA_X3.pem -d -u iotagent -P ${MQTT__iotagent} -t /${DELIVERY_ROBOT_TYPE}/${DELIVERY_ROBOT_02}/cmd -m "{\"send_cmd\":{\"time\":\"$(now)\",\"cmd\":\"navi\",\"waypoints\":[{\"point\":{\"x\":-1.5,\"y\":1.5,\"z\":0},\"angle\":{\"roll\":0,\"pitch\":0,\"yaw\":-1.57}}]}}"

#### confirm mqtt message

example)

```
Client mosq-TX4bePNymjLv3bvrtN received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_02/cmd', ... (159 bytes))
{"send_cmd":{"time":"2019-11-11T12:16:52.974547+09:00","cmd":"navi","waypoints":[{"point":{"x":-1.5,"y":1.5,"z":0},"angle":{"roll":0,"pitch":0,"yaw":-1.57}}]}}
```

#### confirm ros message for delivery_robot_02 ( `/robot_bridge/megarover_01/cmd` )

example)

```yaml
id: "delivery_robot_02"
type: "delivery_robot"
time: "2019-11-11T12:16:52.974547+09:00"
cmd: "navi"
waypoints:
  -
    point:
      x: -1.5
      y: 1.5
      z: 0.0
    angle_optional:
      valid: True
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: -1.57
---
```